In [1]:
#pip install azureml-core

In [2]:
from azureml.core import Workspace
ws = Workspace.from_config(path = "config.json")
print(ws)

Workspace.create(name='MSA-model-deployment', subscription_id='8f613df3-30f7-40d7-a9a3-8861f9369752', resource_group='MSA-model-deployment')


In [3]:
from azureml.core.model import Model

# Register model
#model = Model.register(ws, model_name = "lit-regressor", model_path = "mlpregressor.pkl") #create version 4
#get model information
model = Model(ws, name='lit-regressor')
print(model)

Model(workspace=Workspace.create(name='MSA-model-deployment', subscription_id='8f613df3-30f7-40d7-a9a3-8861f9369752', resource_group='MSA-model-deployment'), name=lit-regressor, id=lit-regressor:4, version=4, tags={}, properties={})


In [4]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig


# Create environment 
env = Environment(name = "lit-regressor")
conda_dep = CondaDependencies()
conda_dep.add_conda_package("numpy")
conda_dep.add_conda_package("xgboost")
conda_dep.add_conda_package("pandas")
conda_dep.add_conda_package("scikit-learn")
conda_dep.add_pip_package("azure-ml-api-sdk")
env.python.conda_dependencies = conda_dep

dummy_inference_config = InferenceConfig(
    environment = env,
    source_directory = "./source_dir",
    entry_script = "./echo_score.py",
)

In [5]:
#pip install azure-ml-api-sdk

In [6]:
from azureml.core.webservice import AciWebservice
aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(
    ws,
    "lit-regressor",
    [model],
    dummy_inference_config,
    aci_config,
    overwrite = True,
)
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-11-03 16:18:48+13:00 Creating Container Registry if not exists.
2021-11-03 16:18:49+13:00 Registering the environment.
2021-11-03 16:18:53+13:00 Use the existing image.
2021-11-03 16:18:54+13:00 Generating deployment configuration..
2021-11-03 16:18:55+13:00 Submitting deployment to compute..
2021-11-03 16:19:01+13:00 Checking the status of deployment lit-regressor..
2021-11-03 16:20:52+13:00 Checking the status of inference endpoint lit-regressor.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [32]:
import requests
import json
import pandas as pd

uri = service.scoring_uri
requests.get(uri)
headers = {"Content-Type": "application/json"}
data = pd.DataFrame.from_dict({"excerpt": ['How are you', 'It is 3 am in the morning', 
                                           'Human will die eventually, only their emotion remains, passed down and be eternal']})
data = data.to_json(orient='columns')
response = requests.post(uri, data = data, headers = headers)
print(response.json())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)